<a href="https://colab.research.google.com/github/mohammedshaneeb-ai/Transfer_Learning_CNN/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle


In [7]:
! cp kaggle.json ~/.kaggle/


In [8]:
! chmod 600 ~/.kaggle/kaggle.json


In [9]:
!kaggle datasets download -d salader/dogs-vs-cats


 98% 1.05G/1.06G [00:09<00:00, 99.1MB/s]
100% 1.06G/1.06G [00:09<00:00, 126MB/s] 


In [10]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [11]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import image_dataset_from_directory


In [12]:
# generating training dataset

train_ds = image_dataset_from_directory(
    directory = './train/',
    labels='inferred',
    label_mode='int',
    image_size = (150,150)
    
) 

Found 20000 files belonging to 2 classes.


In [13]:
# generating  validation dataset

validation_ds = image_dataset_from_directory(
    directory = './test/',
    labels = 'inferred',
    label_mode = 'int',
    image_size = (150,150)
)


Found 5000 files belonging to 2 classes.


In [14]:
conv_base = VGG16(
    weights= 'imagenet',
    include_top= False,
    input_shape= (150,150,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [15]:
def process_image(image,label):
    image = tensorflow.cast(image/255.,tensorflow.float32)
    
    return image,label

train_ds = train_ds.map(process_image)
validation_ds = validation_ds.map(process_image)

In [16]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [17]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [18]:
conv_base.trainable = False

In [19]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(train_ds,epochs=10,batch_size=64,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 75s 100ms/step - loss: 0.2713 - accuracy: 0.8863 - val_loss: 0.2080 - val_accuracy: 0.9112
Epoch 2/10
625/625 [==============================] - 55s 87ms/step - loss: 0.1872 - accuracy: 0.9213 - val_loss: 0.2025 - val_accuracy: 0.9140
Epoch 3/10
625/625 [==============================] - 54s 86ms/step - loss: 0.1639 - accuracy: 0.9331 - val_loss: 0.2222 - val_accuracy: 0.9086
Epoch 4/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1422 - accuracy: 0.9405 - val_loss: 0.2266 - val_accuracy: 0.9056
Epoch 5/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1185 - accuracy: 0.9522 - val_loss: 0.2264 - val_accuracy: 0.9130
Epoch 6/10
625/625 [==============================] - 55s 87ms/step - loss: 0.0921 - accuracy: 0.9644 - val_loss: 0.2539 - val_accuracy: 0.9082
Epoch 7/10
625/625 [==============================] - 64s 102ms/step - loss: 0.0756 - accuracy: 0.9710 - val_loss: 0.2565 - val_accur

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________
